In [1]:
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation, Embedding, TimeDistributed

Using TensorFlow backend.


In [2]:
emotionFile = open("emotionFile.txt","r")
emotion=emotionFile.read()
emotionFile.close()
print(emotion)
#emotion = input("Enter the Emotion\n")
#data_directory = "../Data2/"+emotion+"/"
data_file = emotion+".txt"
charIndex_json = "char_to_index.json"
model_weights_directory = 'Model_Weights_2/'
BATCH_SIZE = 16
SEQ_LENGTH = 64

ecstacy


In [3]:
def make_model(unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (1, 1))) 
    #model.add(Embedding(input_dim = unique_chars, output_dim = 1024, batch_input_shape = (1, 1))) 
  
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    
    model.add(LSTM(256, stateful = True))
    
    
    #remember, that here we haven't given return_sequences = True because here we will give only one character to generate the
    #sequence. In the end, we just have to get one output which is equivalent to getting output at the last time-stamp. So, here
    #in last layer there is no need of giving return sequences = True.
    model.add(Dropout(0.2))
    
    model.add((Dense(38)))
    model.add(Activation("softmax"))
    
    return model

    #downgraded model
    '''
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (BATCH_SIZE, SEQ_LENGTH))) 
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(128, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(unique_chars)))

    model.add(Activation("softmax"))
    
    return model
    '''

In [4]:
def generate_sequence(epoch_num, initial_index, seq_length):
    with open(charIndex_json,'r') as f:
        char_to_index = json.load(f)
    index_to_char = {i:ch for ch, i in char_to_index.items()}
    unique_chars = len(index_to_char)
    model = make_model(unique_chars)
    model.load_weights(model_weights_directory + emotion+"_Weights_20.h5".format(epoch_num))
    sequence_index = [initial_index]
    
    for _ in range(seq_length):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
        predicted_probs = model.predict_on_batch(batch).ravel()
        sample = np.random.choice(range(unique_chars), size = 1, p = predicted_probs)
        
        sequence_index.append(sample[0])
    
    seq = ''.join(index_to_char[c] for c in sequence_index)
    
    cnt = 0
    for i in seq:
        cnt += 1
        if i == "\n":
            break
    seq1 = seq[cnt:]
    #above code is for ignoring the starting string of a generated sequence. This is because we are passing any arbitrary 
    #character to the model for generating music. Now, the model start generating sequence from that character itself which we 
    #have passed, so first few characters before "\n" contains meaningless word. Model start generating the music rhythm from
    #next line onwards. The correct sequence it start generating from next line onwards which we are considering.
    
    cnt = 0
    for i in seq1:
        cnt += 1
        if i == "\n" and seq1[cnt] == "\n":
            break
    seq2 = seq1[:cnt]
    #Now our data contains three newline characters after every tune. So, the model has leart that too. So, above code is used for
    #ignoring all the characters that model has generated after three new line characters. So, here we are considering only one
    #tune of music at a time and finally we are returning it..
    return seq2

In [5]:
#ep = int(input("1. Which epoch number weight you want to load into the model(10, 20, 30, ..., 90). Small number will generate more errors in music: "))
#ar = int(input("\n2. Enter any number between 0 to 86 which will be given as initial charcter to model for generating sequence: "))
#ln = int(input("\n3. Enter the length of music sequence you want to generate. Typical number is between 300-600. Too small number will generate hardly generate any sequence: "))

ep=10
ar=5
ln=600

music = generate_sequence(ep, ar, ln)

print("\nMUSIC SEQUENCE GENERATED: \n")
music = "X:1\nT:sampleMusic\nK:Em\n"+music
print(music)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


MUSIC SEQUENCE GENERATED: 

X:1
T:sampleMusic
K:Em
[c2f2A2G2c2C2][A0A0c0] [e2e2] [d0f0F0d0G,,0][a/e/][F0g0] |
[e2a2d2f2] [d0C0d0] [d2d2e2f2] [d0C0] [c2C2c2] | [C0A0E0] [d0d0f0F20a0f0] [d2e2] [e0C0G0g0G,0G,0E,0C,0] | C,/[f/f/f/][gggg] C,/[f/e/f/][g2g2f2G,2B,2D2G,,2]>[d2e2] | [E0d0C0d0C0A0E0C0E,0G,0G,0A,0G,0]C,, [F,0F,0A,0C,0F,,0^F,0G,0D,0B,0] F,,2 A,,2 [^G,2B,2E,,2F,,2] G,,2[BG,]B, |
[B0D0] F,,2 =G,, F[ A, |
[ADC] dF AE [F2B2c2] G2 | [AD,C]>[B/f/][cg/B,/] [c2C2E2F2] [GCgEECG,,,][ECd] | [A,,AceE,A,,^cc][dA,,] [A0c0f0C0A,0C0c0]< [e0e/][e/e/] [efgg] [g


In [6]:
from selenium import webdriver
driver = webdriver.Chrome("C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe")

driver.get("https://colinhume.com/music.aspx")
input_element = driver.find_element_by_id("InBox")
input_element.send_keys(music)

button_element = driver.find_element_by_id("ConvBtn")
button_element.click()

button_elements = driver.find_elements_by_class_name("Button")
for button_element in button_elements:
    if button_element.text == "Play":
            driver.get(button_element.find_element_by_tag_name("a").get_attribute("href"))
            break

In [39]:

'''

Do the following in Python IDLE


'''



# Import the Gtts module for text  
# to speech conversion 
from gtts import gTTS 
from pydub import AudioSegment
from selenium import webdriver
from time import sleep
import os 
import glob


AudioSegment.converter = "C:\\Program Files\\ffmpeg-20191202-968c4cb-win64-static\\bin\\ffmpeg.exe"
AudioSegment.ffmpeg = "C:\\Program Files\\ffmpeg-20191202-968c4cb-win64-static\\bin\\ffmpeg.exe"
AudioSegment.ffprobe ="C:\\Program Files\\ffmpeg-20191202-968c4cb-win64-static\bin\\ffprobe.exe"


buffer = []
file = open("lyrics.txt","r")
lyrics=file.read()
file.close()
lyrics=lyrics.split("\n")
for line in lyrics:
    buffer.append(line)
multiline_string = "\n".join(buffer)
  
myobj = gTTS(text=multiline_string, lang='en', slow=False) 
  
myobj.save("output_.mp3") 

def get_download_path():
    """Returns the default downloads path for linux or windows"""
    if os.name == 'nt':
        import winreg
        sub_key = r'SOFTWARE\Microsoft\Windows\CurrentVersion\Explorer\Shell Folders'
        downloads_guid = '{374DE290-123F-4565-9164-39C4925E467B}'
        with winreg.OpenKey(winreg.HKEY_CURRENT_USER, sub_key) as key:
            location = winreg.QueryValueEx(key, downloads_guid)[0]
        return location
    else:
        return os.path.join(os.path.expanduser('~'), 'downloads')
print(get_download_path())

sleep(1)

driver = webdriver.Chrome("C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe")

print(glob.glob(get_download_path()+"\*.mid"))
fn=glob.glob(get_download_path()+"\*.mid")
fn=fn[0]

value = fn

driver.get("https://www.onlineconverter.com/midi-to-mp3")
input_element = driver.find_element_by_id("file")
input_element.send_keys(value)

button_element = driver.find_element_by_id("convert-button")
button_element.click()


sleep(20)


url = driver.find_element_by_id("convert-message")
element = url.find_element_by_css_selector("a").get_attribute("href")
print(element)


driver.get(element)

sleep(5)

print(glob.glob(get_download_path()+"\*.mp3"))
fn=glob.glob(get_download_path()+"\*.mp3")
fn=fn[0] #output of the first music file generated
print(fn)

#os.system("start output_.mp3") 
#os.system("start "+fn) 


print('ch1')
sound1 = AudioSegment.from_file("output_.mp3")
print('ch2')

sound2 = AudioSegment.from_file(fn)
print('ch3')
combined = sound1.overlay(sound2)
print('ch4')
combined.export("FinalMusic.mp3", format='mp3')

os.system("start FinalMusic.mp3") 

C:\Users\srika\Downloads


SessionNotCreatedException: Message: session not created
from chrome not reachable
  (Session info: chrome=78.0.3904.108)


In [31]:
fn

'C:\\Users\\srika\\Downloads\\0531291446103.119.35.2.mid'